# MESA Batch Runner

This notebook creates a shell script to execute multiple MESA simulations from a set of batch inlists. The script is designed to work without Python installed on your local machine.

## Instructions

1. Upload your batch inlists zip file (required for verification)
2. Run all cells in sequence
3. Download the generated shell script
4. Place the script in your MESA work directory
5. Run the script on your local Unix/Linux/macOS machine where MESA is installed (in the same lab1/ directory)

In [ ]:
# Import required libraries
import os
import zipfile
import io
from google.colab import files
from IPython.display import display, HTML

## 1. Upload Batch Inlists (Required for Verification)

Upload your batch inlists zip file. This step is required to:

1. Verify you have the correct inlists before generating the run script
2. Confirm the number and names of the inlists that will be processed
3. Ensure the run script will work with your specific inlists

Note: The actual content of these inlists will not be used in creating the run script. The script will look for inlists in the expected directory structure on your local machine when you run it.

**IMPORTANT**: If you created your inlists using the MESA_make_batch notebook, upload the zip file that was generated there.

In [ ]:
print("Please upload your batch_inlists.zip file to verify your inlists before generating the run script.")
uploaded = files.upload()

if not uploaded:
    raise ValueError("No file was uploaded. Please upload your batch inlists zip file to continue.")

zip_filename = list(uploaded.keys())[0]  # Get the filename
print(f"Uploaded: {zip_filename}")

# Extract and list the inlists to verify contents
inlist_names = []
try:
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            if file_info.filename.endswith('.inp'):
                inlist_names.append(os.path.basename(file_info.filename))
except zipfile.BadZipFile:
    raise ValueError("The uploaded file is not a valid zip file. Please upload a properly formatted batch_inlists.zip file.")

if not inlist_names:
    raise ValueError("No .inp files found in the zip file. Please make sure your zip contains properly formatted inlist files.")

print(f"\nFound {len(inlist_names)} inlist files in the zip.")
print("Sample inlist files:")
for name in inlist_names[:5]:  # Show first 5 names
    print(f"  - {name}")
if len(inlist_names) > 5:
    print(f"  ... and {len(inlist_names) - 5} more")

print("\n✅ Verification complete. Your batch inlists are ready for processing.")
print(f"When you run the generated script, it will process {len(inlist_names)} MESA simulations.")

## 2. Create Batch Run Script

Now we'll create a shell script that can be used to run all the MESA simulations without requiring Python on your local machine.

In [ ]:
def create_unix_shell_script():
    """Create a Unix/Linux/macOS shell script to run MESA simulations"""
    unix_script = '''
#!/bin/bash

# MESA Batch Runner
# This script will run MESA for each inlist in batch_runs/batch_inlists

echo "Checking environment..."

# Check if we're in the right directory
if [ ! -f "inlist" ] || [ ! -f "star" ]; then
    echo "WARNING: Moving to working directory."
    cd ../
fi

# Check if we're in the right directory
if [ ! -f "inlist" ] || [ ! -f "star" ]; then
    echo "ERROR: This script must be run from the main MESA work directory OR batch_runs/."
    cd ../
    exit 1
fi

BATCH_DIR="batch_runs/batch_inlists"
OUTPUT_DIR="batch_runs/runs"

# Check if batch inlists exist
if [ ! -d "$BATCH_DIR" ]; then
    echo "Error: Batch directory $BATCH_DIR not found."
    echo "Please create it and add your inlist files."
    exit 1
fi

# Count inlist files
if [ $(ls -1 "$BATCH_DIR"/*.inp 2>/dev/null | wc -l) -eq 0 ]; then
    echo "Error: No inlist files found in $BATCH_DIR"
    exit 1
fi

# Get total number of inlists
TOTAL=$(ls -1 "$BATCH_DIR"/*.inp | wc -l)
echo "Found $TOTAL inlist files in $BATCH_DIR"

# Confirm with user
if [ "$1" != "--force" ]; then
    echo
    echo "You are about to run $TOTAL MESA simulations."
    echo "This might take a significant amount of time and computational resources."
    if [ $TOTAL -gt 10 ]; then
        echo "WARNING: Running $TOTAL simulations could take hours or even days!"
    fi
    echo
    read -p "Do you want to continue? (yes/no): " response
    
    if [[ ! "$response" =~ ^[Yy][Ee]?[Ss]?$ ]]; then
        echo "Batch run cancelled."
        exit 0
    fi
fi

# Create output directory if it doesn't exist
mkdir -p "$OUTPUT_DIR"

# Process each inlist
CURRENT=0
for inlist_file in "$BATCH_DIR"/*.inp; do
    CURRENT=$((CURRENT + 1))
    inlist_name=$(basename "$inlist_file" .inp)
    run_dir="$OUTPUT_DIR/$inlist_name"
    
    echo "[$CURRENT/$TOTAL] Processing $inlist_name..."
    
    # Create run directory and subdirectories
    mkdir -p "$run_dir"
    mkdir -p "$run_dir/LOGS"
    mkdir -p "$run_dir/photos"
    
    # Copy inlist file
    cp -f "$inlist_file" inlist_project
    
    # Run MESA
    echo "Running MESA with $inlist_name parameters..."
    start_time=$(date +%s)
    
    ./star > "$run_dir/run.log" 2>&1
    
    end_time=$(date +%s)
    elapsed=$((end_time - start_time))
    echo "Run completed in $elapsed seconds."
    
    # Copy results
    if [ -d "LOGS" ]; then
        cp -f LOGS/* "$run_dir/LOGS/" 2>/dev/null
    fi
    
    if [ -d "photos" ]; then
        cp -f photos/* "$run_dir/photos/" 2>/dev/null
    fi
    
    cp -f inlist_project "$run_dir/"
    cp -f inlist "$run_dir/"
    if [ -f "inlist_pgstar" ]; then
        cp -f inlist_pgstar "$run_dir/"
    fi
    
    # Find and copy model file if it exists
    model_file=$(grep "save_model_filename" "$inlist_file" | sed "s/.*save_model_filename = '\([^']*\)'.*/\1/")
    if [ -f "$model_file" ]; then
        cp -f "$model_file" "$run_dir/"
    fi
    
    echo "Completed run for $inlist_name"
    echo "------------------------------------------"
done

echo "All batch runs completed!"
echo
'''
    
    return unix_script.strip()

In [ ]:
# Generate the shell script
script_content = create_unix_shell_script()
script_filename = 'run_batch_collab.sh'

# Write the script to a file
with open(script_filename, 'w') as f:
    f.write(script_content)

# Make it executable
!chmod +x {script_filename}

print(f"Created {script_filename}")
print(f"Script is ready to process {len(inlist_names)} MESA simulations")

## 3. Download the Run Script

Download the script to use in your MESA work directory.

In [ ]:
files.download(script_filename)

## 4. Installation and Usage Instructions

### IMPORTANT: How to Use the Generated Script

After downloading the script, follow these steps to run your MESA simulations:

1. Make sure you have extracted the batch inlists to the `batch_runs/batch_inlists` directory in your MESA work directory
   - The extracted directory structure should match what was in your verified zip file
   - The script will look for `.inp` files in this directory

2. Place the downloaded script in your MESA work directory (the main directory with your MESA installation) or in the `batch_runs` directory

3. Open Terminal and navigate to your MESA work directory or the `batch_runs` directory

4. Make sure the script is executable:
   ```bash
   chmod +x run_batch_collab.sh
   ```

5. Run the script:
   ```bash
   ./run_batch_collab.sh
   ```

6. To skip the confirmation prompt, use:
   ```bash
   ./run_batch_collab.sh --force
   ```

### Expected Output

The script will:

1. Run each inlist sequentially
2. Create a separate output directory for each run in `batch_runs/runs`
3. Copy all relevant output files to these directories
4. Show progress as it runs

### Resource Considerations

* MESA simulations can be computationally intensive
* Running many simulations may take hours or days depending on your hardware
* Consider running a subset of simulations initially to test

### Directory Structure

After running, your MESA directory structure should look like:

```
mesa_work_directory/
├── batch_runs/
│   ├── batch_inlists/
│   │   ├── inlist_M2_Z0.0140_noovs.inp
│   │   ├── inlist_M2_Z0.0140_exponential_fov0.01_f00.001.inp
│   │   └── ... (other inlist files)
│   ├── run_batch_collab.sh (if placed here)
│   └── runs/
│       ├── inlist_M2_Z0.0140_noovs/
│       │   ├── LOGS/
│       │   ├── photos/
│       │   ├── inlist
│       │   ├── inlist_project
│       │   ├── inlist_pgstar
│       │   ├── run.log
│       │   └── M2_Z0.0140_noovs.mod
│       └── ... (other run directories)
├── inlist
├── inlist_project
├── inlist_pgstar
├── run_batch_collab.sh (if placed here)
└── ... (other MESA files)
```